In [1]:
import json
import re
from datetime import datetime
import random
import nltk
import pickle
from nameparser.parser import HumanName
from nltk.corpus import wordnet
# import names
import gender_guesser.detector as gender
import calendar

In [4]:
with open("data/consolidatedData.json", 'r', encoding='utf-8') as f:
    consolidatedData = json.load(f)

p = random.choice(consolidatedData['allPrunedParaComplete'])

In [7]:
p

"[BAND_NAME] (in its last year of existence Shit & Chalou) was a [GENRE] band in pop rock and folk rock established in [YEAR]. The Aarhus-based band was made up of Anne Linnet, [PERSON_NAME_FULL_3_FEMALE], Lis Sørensen, [PERSON_NAME_FULL_0_FEMALE] and [PERSON_NAME_FULL_1_FEMALE].\nIn the early [YEAR]s, Anne Linnet used to be in the band Tears along with her husband [PERSON_NAME_FULL_4_MALE]. [PERSON_NAME_FIRST_4] had the idea of a girl band and suggested two recruits from Århus Friskole (Århus Free School), namely [PERSON_NAME_FIRST_3] [PERSON_NAME_LAST_5] and [PERSON_NAME_FULL_1_FEMALE] to join Anne to form a band. Soon Lis Sørensen also joined in. They practiced at [PERSON_NAME_FIRST_1]'s parents house near Brabrand Lake in Aarhus.\nIn [YEAR] there they took part in women-only Kvindefestival i Fælledparken in Copenhagen. After black American activist [PERSON_NAME_FULL_2_FEMALE] spoke, [BAND_NAME] came on stage with an accompanying male-band Delta Blues Band playing for them, which im

## Replace Band Name

In [4]:
with open('titles', 'rb') as inf:
    titles = pickle.load(inf)
    
determiners = []
nouns = []
adjectives = []
for title in titles:
    tagged_title = nltk.pos_tag(nltk.word_tokenize(title.lower()))
    for tagged_word in tagged_title:
        word = tagged_word[0]
        pos = tagged_word[1]
        if pos == 'DT':
            determiners.append(word)
        elif pos == 'NN' or pos == 'NNS':
            nouns.append(word.capitalize())
        elif pos == 'JJ':
            adjectives.append(word.capitalize())

In [5]:
def replace_band_name(text):
    def getRandName():
        determiner = random.choice(determiners).capitalize()
        [adjective1, adjective2] = random.sample(adjectives, 2)
        [noun1, noun2] = random.sample(nouns, 2)

        title_format = random.randrange(3)
        if title_format == 0:
            return determiner + ' ' + adjective1 + ' ' + noun1
        elif title_format == 1:
            return determiner + ' ' + adjective1 + ' ' + adjective2 + ' ' + noun1
        elif title_format == 2:
            return determiner + ' ' + noun1 + ' and ' + determiner + ' ' + noun2
        
    new_name = getRandName()
    text = re.sub('\[BAND_NAME\]', new_name, text)
    return text

## Replace Years

In [6]:
def replace_years(text):
    num_years = text.count('[YEAR]')
    first_year = datetime.now().year - (5 * num_years)
    years = [first_year]
    for i in range(1, num_years):
        years.append(years[i-1] + random.randint(0, 5))
        
    j = -1
    def get_year(matchobj):
        nonlocal j
        j += 1
        return years[j]

    return re.sub("\[YEAR\]", lambda x: str(get_year(x)), text)

## Replace Months

In [7]:
def replace_months(text):
    months = [calendar.month_name[i] for i in range(1,13)] + [calendar.month_abbr[i] for i in range(1,13)]
    return re.sub('\[MONTH\]', lambda x: random.choice(months), text)

In [13]:
replace_months("s replacement later that year.[MONTH]")

's replacement later that year.February'

## Replace Names (doesn't work currently)
https://stackoverflow.com/questions/20290870/improving-the-extraction-of-human-names-with-nltk
Shivansh bhandari's answer

In [8]:
person_list = []
person_names=person_list
def get_human_names(text):
    tokens = nltk.tokenize.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    sentt = nltk.ne_chunk(pos, binary = False)

    person = []
    name = ""
    for subtree in sentt.subtrees(filter=lambda t: t.label() == 'PERSON'):
        for leaf in subtree.leaves():
            person.append(leaf[0])
        if len(person) > 1: #avoid grabbing lone surnames
            for part in person:
                name += part + ' '
            if name[:-1] not in person_list:
                person_list.append(name[:-1])
            name = ''
        person = []
#     print (person_list)

names_ = get_human_names(text)
for person in person_list:
    person_split = person.split(" ")
    for name in person_split:
        if wordnet.synsets(name):
            if(name in person):
                person_names.remove(person)
                break

print(person_names)

NameError: name 'text' is not defined

TODO: function to replace names with autogen names
maybe create dict that matches old to new
also need to replace last names as well as first names (maybe first names too?)
i think can replace all full names, then can look for last or first because shouldn't be an issue that we already replaced some

In [24]:
name_genders = {}
d = gender.Detector()
for name in person_names:
    g = d.get_gender(name.split()[0])
    print(name, g)
    if g == 'male' or g == 'female':
        name_genders[name] = g
name_genders

Stefanie Sargent female
Valerie Agnew female
Ben London male
Sick 'Em unknown
Roisin Dunne female
Home Alive unknown
Miami Beach unknown


{'Stefanie Sargent': 'female',
 'Valerie Agnew': 'female',
 'Ben London': 'male',
 'Roisin Dunne': 'female'}

In [25]:
name_replacements = {}
for k,v in name_genders.items():
    name_replacements[k] = names.get_full_name(gender=v)
name_replacements

{'Stefanie Sargent': 'Danielle Grady',
 'Valerie Agnew': 'Lucy Nelson',
 'Ben London': 'David Gambino',
 'Roisin Dunne': 'Emily Dancy'}

In [26]:
for k,v in name_replacements.items():
    text = re.sub(k.split()[0], v.split()[0], text) # replace first name
    text = re.sub(k.split()[1], v.split()[1], text) # replace last name

## Replace Genre

### Get all Genres

In [9]:
with open('data/consolidatedData.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
genres = set()
for genre in data['allGenres']:
    if isinstance(genre, str):
        g = re.split(',|\[', genre)[0]
        genres.add(g)
        
genres = list(genres) # need to convert to list to be able to take a random choice from it

In [10]:
def replace_genre(text):
    return re.sub('\[GENRE\]', random.choice(genres), text)

## Replace All Placeholders

In [14]:
with open('data/consolidatedData.json', 'r', encoding='utf-8') as inf:
    paras = json.load(inf)

#with open('data/0.json', 'r', encoding='utf-8') as inf:
#    data = json.load(inf)
    
para = paras['allPrunedParaComplete'][3]
print(para)
print("")
para = replace_band_name(para)
para = replace_years(para)
para = replace_months(para)
para = replace_genre(para)
print(para)

[BAND_NAME] is an [GENRE] band formed in [YEAR] in Los Angeles, California, by five musicians: three from Mississippi, one from Pennsylvania and one from Indiana. The band currently consists of guitarists Rogers Stevens and [PERSON_NAME_FULL_0_MALE], drummer Glen Graham, vocalist [PERSON_NAME_FULL_4_MALE] and bassist [PERSON_NAME_FULL_3_MALE]. They are best known for their [YEAR] hit "No Rain", and enjoyed critical and commercial success in the early [YEAR]s with their neo-psychedelic take on alternative rock.
[BAND_NAME] released two albums on Capitol Records – [BAND_NAME] ([YEAR]) and Soup ([YEAR]) – before their original lead vocalist [PERSON_NAME_FULL_2_FEMALE] died of a drug overdose on the band's tour bus in New Orleans on [MONTH] 21, [YEAR]. After four years of unsuccessfully searching for a replacement for [PERSON_NAME_LAST_2], [BAND_NAME] officially dissolved in [YEAR]. The remaining members reformed the band in [YEAR] with [PERSON_NAME_LAST_4] and recorded one album with him,